## Fine Tune Gemma 2 2b quantized model on V3 Dataset

In [1]:
# installing unsloth

%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",
    "unsloth/gemma-2-2b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.7: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

## Adding LORA Adapters

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.7 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


## Dataset Preparation

Here using Roman Nepali Fine Tune Dataset for Instruction Fine Tuning.

NOTE: **EOS_TOKEN** must be added to the tokenized output otherwise we will get the infinite generation.

In [4]:
from datasets import load_dataset

In [5]:
dataset = load_dataset("manojbaniya/roman-nepali-ift", split="train")

dataset

README.md:   0%|          | 0.00/222 [00:00<?, ?B/s]

v3_fine_tune.csv:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16614 [00:00<?, ? examples/s]

Dataset({
    features: ['category', 'instruction', 'context', 'question', 'response'],
    num_rows: 16614
})

In [6]:
dataset[100]

{'category': 'translate',
 'instruction': 'Translate the text below from Nepali to English:',
 'context': ' ',
 'question': 'Rog rok ra swasthya badhauna ma sabhik swasthya prakriyako mahattva avashyak cha.',
 'response': 'The importance of public health initiatives in preventing diseases and promoting well-being is crucial.'}

## Prompt Template

In [55]:
prompt_template = """
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
{instruction}
{question}
<CONTEXT>
{context}
<RESPONSE>
{response}
"""

In [56]:
prompt = prompt_template.format(instruction="Translate the text from english to roman nepali", question="I Love you", context = "", response = "Ma timilai maya garxu")

In [57]:
print(prompt)


Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Translate the text from english to roman nepali
I Love you
<CONTEXT>

<RESPONSE>
Ma timilai maya garxu



In [58]:
EOS_TOKEN = tokenizer.eos_token

In [59]:
def formatting_prompt(examples):
  instructions = examples["instruction"]
  questions = examples["question"]
  contexts = examples["context"]
  responses = examples["response"]

  prompts = []

  for instruction, question, context, response in zip(instructions, questions, contexts, responses):
    context = "null" if context is None else context
    instruction = "" if instruction is None else instruction

    prompt = prompt_template.format_map({
        "instruction": instruction,
        "question": question,
        "context": context,
        "response": response
    })
    prompt = prompt + EOS_TOKEN
    prompts.append(prompt)

  return {"prompts": prompts}

## Formatting Dataset

In [60]:
dataset = dataset.map(formatting_prompt, batched=True)

Map:   0%|          | 0/16614 [00:00<?, ? examples/s]

In [61]:
print(dataset[100]["prompts"])


Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Translate the text below from Nepali to English:
Rog rok ra swasthya badhauna ma sabhik swasthya prakriyako mahattva avashyak cha.
<CONTEXT>
 
<RESPONSE>
The importance of public health initiatives in preventing diseases and promoting well-being is crucial.
<eos>


In [62]:
print(dataset[1000]["prompts"])


Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Translate the text below from English to Nepali:
I have been trying to limit my use of plastic to help protect the environment.
<CONTEXT>
 
<RESPONSE>
Maile paryavaran ko raksha garna plastic ko prayog simita garna khojiraheko chu.
<eos>


In [63]:
print(dataset[9000]["prompts"])


Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Summarize the following text:
Ek group of students le plastic-free campaign start garcha. Aba unko efforts le school ra community ma awareness increase garcha.
<CONTEXT>
null
<RESPONSE>
Students ko plastic-free campaign le school ra community ma awareness spread garcha.
<eos>


In [64]:
print(dataset[12000]["prompts"])


Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
iPhone ko price kati ho?
<CONTEXT>
Product List: Redmi Note 9 Pro, Price: Rs. 20000. Samsung Galaxy, Price: Rs. 60000. iPhone, Price: Rs. 100000. Calculator, Price: Rs. 500. Sony TV, Price: Rs. 40000.
<RESPONSE>
iPhone ko price Rs. 100000 ho.
<eos>


## Training the Model

In [65]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [66]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "prompts",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/16614 [00:00<?, ? examples/s]

In [67]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.697 GB of memory reserved.


In [68]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 16,614 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
1,3.561000
2,4.024400
3,3.825800
4,3.749600
5,3.308500
6,3.182300
7,2.806800
8,2.711100
9,1.808900
10,2.054200


## Train stat detail

In [69]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

211.041 seconds used for training.
3.52 minutes used for training.
Peak reserved memory = 4.953 GB.
Peak reserved memory for training = 2.256 GB.
Peak reserved memory % of max memory = 33.584 %.
Peak reserved memory for training % of max memory = 15.297 %.


## Inference

In [70]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [79]:
inputs = prompt_template.format(instruction="Translate the text from english to roman nepali", question="I am happy today", context = "", response = "")

In [80]:
inputs = tokenizer([inputs], return_tensors="pt").to("cuda")

In [81]:
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
response = tokenizer.batch_decode(outputs)

In [82]:
print(response[0])

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Translate the text from english to roman nepali
I am happy today
<CONTEXT>

<RESPONSE>

Maile din ekdam happy chha.
<eos>


In [86]:
def generate_response(instruction, context, question):
  inputs = prompt_template.format(instruction=instruction, question=question, context=context, response="")
  inputs = tokenizer([inputs], return_tensors="pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
  response = tokenizer.batch_decode(outputs)
  return response[0]

### RAG on Store Information

In [87]:
print(generate_response(
    "Answer the following question by looking at the context if context is given.",
    "Hamro platform ma NIC Asia, Nabil Bank, Global IME, ra arko dherai mobile banking apps le support garcha.",
    "mobile banking ko kun kun bank apps support huncha?" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
mobile banking ko kun kun bank apps support huncha?
<CONTEXT>
Hamro platform ma NIC Asia, Nabil Bank, Global IME, ra arko dherai mobile banking apps le support garcha.
<RESPONSE>

Mobile banking apps le NIC Asia, Nabil Bank, Global IME, ra arko dherai bank apps support garcha.
<eos>


In [88]:
print(generate_response(
    "Answer the following question by looking at the context if context is given.",
    "Store ma online order ko subidha cha, jasle mobile app ra website bata order lina sakcha. Minimum order Rs. 500 hunu parchha. Delivery Kathmandu upatyaka bhitra matra available cha. Online payment ya Cash on Delivery (COD) ko option cha.",
    "Webiste baata order garna milxa?" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
Webiste baata order garna milxa?
<CONTEXT>
Store ma online order ko subidha cha, jasle mobile app ra website bata order lina sakcha. Minimum order Rs. 500 hunu parchha. Delivery Kathmandu upatyaka bhitra matra available cha. Online payment ya Cash on Delivery (COD) ko option cha.
<RESPONSE>

Website ma order garna milxa. Minimum order Rs. 500 hunu parchha. Delivery Kathmandu upatyaka bhitra matra available cha.
<eos>


In [89]:
print(generate_response(
    instruction="Answer the following question by looking at the context if context is given.",
    context="Hamro store ko name All Electronics store ho yo Dharan ma xa Hamro store bihan 10 baje dekhi rati 6 baje samma khulxa.",
    question="store ko name ke ho" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
store ko name ke ho
<CONTEXT>
Hamro store ko name All Electronics store ho yo Dharan ma xa Hamro store bihan 10 baje dekhi rati 6 baje samma khulxa.
<RESPONSE>

All Electronics store ho.
<eos>


In [90]:
print(generate_response(
    instruction="Answer the following question by looking at the context if context is given.",
    context="Hamro store ko name All Electronics store ho yo Dharan ma xa Hamro store bihan 10 baje dekhi rati 6 baje samma khulxa.",
    question="store kaha xa?" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
store kaha xa?
<CONTEXT>
Hamro store ko name All Electronics store ho yo Dharan ma xa Hamro store bihan 10 baje dekhi rati 6 baje samma khulxa.
<RESPONSE>

Hamro store ko name All Electronics store ho yo Dharan ma xa.
<eos>


In [92]:
print(generate_response(
    instruction="Answer the following question by looking at the context if context is given.",
    context="Hamro store ko name All Electronics store ho yo Dharan ma xa Hamro store bihan 10 baje dekhi rati 6 baje samma khulxa.",
    question="kati baje dekhi store khulla hunxa?" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
kati baje dekhi store khulla hunxa?
<CONTEXT>
Hamro store ko name All Electronics store ho yo Dharan ma xa Hamro store bihan 10 baje dekhi rati 6 baje samma khulxa.
<RESPONSE>

Hamro store ko name All Electronics store ho yo Dharan ma xa Hamro store ko khulna baje 10 baje ho.
<eos>


### Test on Product Inquiry

In [93]:
print(generate_response(
    instruction="Answer the following question by looking at the context if context is given.",
    context="Product detail: Redmi Note 9 pro Price Rs. 10000, stock: Yes, Color: Green, Blue IPhone 11 pro, Price Rs.110000, stock: No, Color: Black, Gray",
    question="Redmi note 9 ko price kati ho?" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
Redmi note 9 ko price kati ho?
<CONTEXT>
Product detail: Redmi Note 9 pro Price Rs. 10000, stock: Yes, Color: Green, Blue IPhone 11 pro, Price Rs.110000, stock: No, Color: Black, Gray
<RESPONSE>

Redmi note 9 ko price Rs. 10000 ho.
<eos>


In [94]:
print(generate_response(
    instruction="Answer the following question by looking at the context if context is given.",
    context="Product detail: Redmi Note 9 pro Price Rs. 10000, stock: Yes, Color: Green, Blue IPhone 11 pro, Price Rs.110000, stock: No, Color: Black, Gray",
    question="Iphone 11 pro ko price kati ho?" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
Iphone 11 pro ko price kati ho?
<CONTEXT>
Product detail: Redmi Note 9 pro Price Rs. 10000, stock: Yes, Color: Green, Blue IPhone 11 pro, Price Rs.110000, stock: No, Color: Black, Gray
<RESPONSE>

Iphone 11 pro ko price Rs.110000 ho.
<eos>


In [95]:
print(generate_response(
    instruction="Answer the following question by looking at the context if context is given.",
    context="Product detail: Redmi Note 9 pro Price Rs. 10000, stock: Yes, Color: Green, Blue IPhone 11 pro, Price Rs.110000, stock: No, Color: Black, Gray",
    question="Redmi Note 9 available xa ki xaina?" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
Redmi Note 9 available xa ki xaina?
<CONTEXT>
Product detail: Redmi Note 9 pro Price Rs. 10000, stock: Yes, Color: Green, Blue IPhone 11 pro, Price Rs.110000, stock: No, Color: Black, Gray
<RESPONSE>

Redmi Note 9 pro available xa, price Rs. 10000, stock: Yes, color: Green, Blue.
<eos>


In [98]:
print(generate_response(
    instruction="Answer the following question by looking at the context if context is given.",
    context="Product detail: Redmi Note 9 pro Price Rs. 10000, stock: Yes, Color: Green, Blue IPhone 11 pro, Price Rs.110000, stock: No, Color: Black, Gray",
    question="Is I phone 11 pro available?" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
Is I phone 11 pro available?
<CONTEXT>
Product detail: Redmi Note 9 pro Price Rs. 10000, stock: Yes, Color: Green, Blue IPhone 11 pro, Price Rs.110000, stock: No, Color: Black, Gray
<RESPONSE>

I phone 11 pro available cha?
<eos>


In [118]:
print(generate_response(
    instruction="Answer the following question by looking at the context if context is given.",
    context="Hamro platform le premium electronics jastai laptops, mobile phones, ra accessories ko reliable collection provide garcha. Fast delivery service ma 48 ghantabitra saman pauna milcha. Payment options ma Khalti, eSewa, debit card, ra cash on delivery cha.",
    question="payment garna k baata garna milxa?" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
payment garna k baata garna milxa?
<CONTEXT>
Hamro platform le premium electronics jastai laptops, mobile phones, ra accessories ko reliable collection provide garcha. Fast delivery service ma 48 ghantabitra saman pauna milcha. Payment options ma Khalti, eSewa, debit card, ra cash on delivery cha.
<RESPONSE>

Payment options ma Khalti, eSewa, debit card, ra cash on delivery cha.
<eos>


In [119]:
print(generate_response(
    instruction="Answer the following question by looking at the context if context is given.",
    context="Hamro platform le premium electronics jastai laptops, mobile phones, ra accessories ko reliable collection provide garcha. Fast delivery service ma 48 ghantabitra saman pauna milcha. Payment options ma Khalti, eSewa, debit card, ra cash on delivery cha.",
    question="store ma k k available xa" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
store ma k k available xa
<CONTEXT>
Hamro platform le premium electronics jastai laptops, mobile phones, ra accessories ko reliable collection provide garcha. Fast delivery service ma 48 ghantabitra saman pauna milcha. Payment options ma Khalti, eSewa, debit card, ra cash on delivery cha.
<RESPONSE>

Store ma laptops, mobile phones, ra accessories available cha.
<eos>


In [104]:
print(generate_response(
    instruction="Answer the following question by looking at the context if context is given.",
    context="Hajur, tapai le similar product haru ko price compare garna sakinchha. Compare button click garera price herna sakinchha. Compare garda tapai le product ko features, rating, ra price difference herna sakchha.",
    question="Store ma product ko price compare garna sakincha?" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Answer the following question by looking at the context if context is given.
Store ma product ko price compare garna sakincha?
<CONTEXT>
Hajur, tapai le similar product haru ko price compare garna sakinchha. Compare button click garera price herna sakinchha. Compare garda tapai le product ko features, rating, ra price difference herna sakchha.
<RESPONSE>

Store ma product ko price compare garna sakincha.
<eos>


### Test on Translation

In [107]:
print(generate_response(
    instruction="Translate the text below from Nepali to English:",
    context="",
    question="aaja sanibar ho" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Translate the text below from Nepali to English:
aaja sanibar ho
<CONTEXT>

<RESPONSE>

I'm going to the bathroom.
<eos>


In [110]:
print(generate_response(
    instruction="Translate the text below from Nepali to English:",
    context="",
    question="India ko prime minister Modi ho" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Translate the text below from Nepali to English:
India ko prime minister Modi ho
<CONTEXT>

<RESPONSE>

India's prime minister is Modi.
<eos>


In [111]:
print(generate_response(
    instruction="Translate the text below from Nepali to English:",
    context="",
    question="Nepal le world cup jityo" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Translate the text below from Nepali to English:
Nepal le world cup jityo
<CONTEXT>

<RESPONSE>

Nepal won the World Cup.
<eos>


In [112]:
print(generate_response(
    instruction="Translate the text below from Nepali to English:",
    context="",
    question="success pauna hami sabai ek jut hunu parxa" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
Translate the text below from Nepali to English:
success pauna hami sabai ek jut hunu parxa
<CONTEXT>

<RESPONSE>

I feel like I'm on the right track.
<eos>


### General Instructions

In [114]:
print(generate_response(
    instruction="You are a helpful assistant. Answer the given user question.",
    context="",
    question="Nepal kun continent ma parxa" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
You are a helpful assistant. Answer the given user question.
Nepal kun continent ma parxa
<CONTEXT>

<RESPONSE>

Nepal kun continent ma Asia ma parxa.
<eos>


In [116]:
print(generate_response(
    instruction="You are a helpful assistant. Answer the given user question.",
    context="",
    question="kasari mitho khana banaune" ))

<bos>
Below is an instruction that describes a task paried with input that provides further context. Write a response that appropriately complete the request.

<INSTRUCTION>
You are a helpful assistant. Answer the given user question.
kasari mitho khana banaune
<CONTEXT>

<RESPONSE>

Mithai banaune ko lagi, ingredients ko list bata saknuhuncha.
<eos>
